# Modelo de Machine Learning

In [1]:
import pandas as pd
import os
from random import randint
import mysql.connector
from tqdm import tqdm

# Ruta de archivos

In [7]:
path_train = os.path.join(os.getcwd(), '..', '4_modeling_module',"TRAIN_DATA")
path_dwh = os.path.join(os.getcwd(), '..', '3_data_preprocesing_module',"DATAWAREHOUSE")
path_scoring_db = os.path.join(os.getcwd(),"DATABASE_SCORING")
path_scored_new_data = os.path.join(os.getcwd(), '..', '6_deploying_module',"DB_SC")
path_dwh_sc = os.path.join(os.getcwd(), '..', '6_deploying_module',"DWH-SC")
path_dwh_final = os.path.join(os.getcwd(), '..', '6_deploying_module',"DATAWAREHOUSE_DP")

In [8]:
df_razon_social = pd.read_csv(os.path.join(path_dwh,'dim_razon_social.csv'))
df_fecha = pd.read_csv(os.path.join(path_dwh,'dim_fecha.csv'))
df_curso_perfil = pd.read_csv(os.path.join(path_dwh,'dim_curso_perfil.csv'))

In [9]:
df_ci_cp = pd.read_csv(os.path.join(path_dwh,'fact_ci_cp.csv'))
df_ci_rs = pd.read_csv(os.path.join(path_dwh,'fact_ci_rs.csv'))
df_oc_cp = pd.read_csv(os.path.join(path_dwh,'fact_oc_cp.csv'))
df_oc_rs = pd.read_csv(os.path.join(path_dwh,'fact_oc_rs.csv'))
df_oec_cp = pd.read_csv(os.path.join(path_dwh,'fact_oec_cp.csv'))
df_oec_rs = pd.read_csv(os.path.join(path_dwh,'fact_oec_rs.csv'))

In [10]:
df_fecha = pd.read_csv(os.path.join(path_dwh,'dim_fecha.csv'))

In [8]:
df_fecha.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2142 entries, 0 to 2141
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id_fecha  2142 non-null   int64 
 1   fecha     2142 non-null   object
 2   dia       2142 non-null   int64 
 3   mes       2142 non-null   int64 
 4   anio      2142 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 83.8+ KB


In [29]:
df_fecha.tail()

,id_fecha,fecha,dia,mes,anio
2137,2137,31-10-2017,31,10,2017
2138,2138,31-10-2018,31,10,2018
2139,2139,31-10-2019,31,10,2019
2140,2140,31-10-2020,31,10,2020
2141,2141,31-10-2021,31,10,2021


# Generación de datos ficticios

Generación de nuevas fechas

In [90]:
nueva_fecha = [
    [2142,'31-01-2023',31,1,2023],
    [2143,'28-02-2023',28,2,2023]
]

In [91]:
df_new_fecha = pd.DataFrame(data=nueva_fecha,columns=df_fecha.columns)

In [92]:
df_fecha = pd.concat([df_fecha,df_new_fecha],ignore_index=True)

## CI CP

In [119]:

new_data_ci_cp = [] 
size_ci_cp = df_ci_cp.shape[0]
size_new_fecha =df_new_fecha.shape[0]
for i in range(300):
    rd_idx = randint(0,size_ci_cp-1)
    rd_f_idx = randint(0,size_new_fecha-1)
    id_rs_rd = df_ci_cp['id_razon_social'].iloc[rd_idx]
    id_cp_rd = df_ci_cp['id_curso_perfil'].iloc[rd_idx]
    id_fecha_rd = df_new_fecha['id_fecha'].iloc[rd_f_idx]
    rd_num_cap = 0
    rd_total_horas = 0
    rd_total_ganancias = 0
    new_data_ci_cp.append([id_rs_rd,id_cp_rd,id_fecha_rd,rd_num_cap,rd_total_horas,rd_total_ganancias])
    
   
df_new_ci_cp = pd.DataFrame(data=new_data_ci_cp,columns=['id_razon_social',	'id_curso_perfil',	'id_fecha',	'num_cap',	'total_horas',	'total_ganancias'])
    

In [120]:
df_new_ci_cp = df_new_ci_cp.drop_duplicates()

In [121]:
df_new_ci_cp['id_fact_ci_cp'] = [x for x in range(df_ci_cp.shape[0],df_ci_cp.shape[0]+df_new_ci_cp.shape[0])]

In [122]:
df_new_ci_cp.insert(0, 'id_fact_ci_cp', df_new_ci_cp.pop('id_fact_ci_cp'))

In [132]:
df_ci_cp = pd.concat([df_ci_cp,df_new_ci_cp],ignore_index=True)

In [125]:
data_ci_cp = pd.merge(data_ci_cp,df_curso_perfil,on='id_curso_perfil',how='inner')
data_ci_cp = pd.merge(data_ci_cp,df_fecha,on='id_fecha',how='inner')
data_ci_cp = pd.merge(data_ci_cp,df_razon_social,on='id_razon_social',how='inner')

In [126]:
data_ci_cp = data_ci_cp.drop(['num_cap','total_horas','total_ganancias'],axis=1)

In [128]:
data_ci_cp.to_csv(os.path.join(path_scoring_db,'data_ci_cp_sc.csv'),index=False)

## CI RS

In [158]:
df_new_ci_rs = df_new_ci_cp.groupby(['id_razon_social','id_fecha']).count().reset_index()[['id_razon_social','id_fecha']]

In [159]:
df_new_ci_rs['id_fact_ci_rs'] = [x for x in range(df_ci_rs.shape[0],df_ci_rs.shape[0]+df_new_ci_rs.shape[0])]
df_new_ci_rs.insert(0, 'id_fact_ci_rs', df_new_ci_rs.pop('id_fact_ci_rs'))
data_ci_rs = pd.concat([df_ci_rs,df_new_ci_rs],ignore_index=True)

In [160]:
data_ci_rs = data_ci_rs.drop(['volumen_cap','volumen_horas','volumen_ganancias','total_cursos'],axis=1)

In [162]:
data_ci_rs = pd.merge(data_ci_rs,df_fecha,on='id_fecha',how='inner')
data_ci_rs = pd.merge(data_ci_rs,df_razon_social,on='id_razon_social',how='inner')

In [176]:
data_ci_rs.to_csv(os.path.join(path_scoring_db,'data_ci_rs_sc.csv'),index=False)

## OC CP 

In [189]:
new_data_oc_cp = [] 
size_oc_cp = df_oc_cp.shape[0]
for i in range(300):
    rd_idx = randint(0,size_ci_cp-1)
    rd_f_idx = randint(0,size_new_fecha-1)
    id_rs_rd = df_oc_cp['id_razon_social'].iloc[rd_idx]
    id_cp_rd = df_oc_cp['id_curso_perfil'].iloc[rd_idx]
    id_fecha_rd = df_new_fecha['id_fecha'].iloc[rd_f_idx]
    new_data_oc_cp.append([id_rs_rd,id_cp_rd,id_fecha_rd])
    
   
df_new_oc_cp = pd.DataFrame(data=new_data_oc_cp,columns=['id_razon_social',	'id_curso_perfil',	'id_fecha'])
    

In [190]:
df_new_oc_cp = df_new_oc_cp.drop_duplicates()
df_new_oc_cp['id_fact_oc_cp'] = [x for x in range(df_oc_cp.shape[0],df_oc_cp.shape[0]+df_new_oc_cp.shape[0])]
df_new_oc_cp.insert(0, 'id_fact_oc_cp', df_new_oc_cp.pop('id_fact_oc_cp'))
data_oc_cp = pd.concat([df_oc_cp,df_new_oc_cp],ignore_index=True)

In [191]:
data_oc_cp.dropna(axis='columns',inplace=True)

In [192]:
data_oc_cp = pd.merge(data_oc_cp,df_curso_perfil,on='id_curso_perfil',how='inner')
data_oc_cp = pd.merge(data_oc_cp,df_fecha,on='id_fecha',how='inner')
data_oc_cp = pd.merge(data_oc_cp,df_razon_social,on='id_razon_social',how='inner')

In [193]:
data_oc_cp.to_csv(os.path.join(path_scoring_db,'data_oc_cp_sc.csv'),index=False)

## OC RS

In [233]:
df_new_oc_rs = df_new_oc_cp.groupby(['id_razon_social','id_fecha']).count().reset_index()
df_new_oc_rs['id_fact_oc_rs'] = [x for x in range(df_oc_rs.shape[0],df_oc_rs.shape[0]+df_new_oc_rs.shape[0])]
df_new_oc_rs.insert(0, 'id_fact_oc_rs', df_new_oc_rs.pop('id_fact_oc_rs'))
data_oc_rs = pd.concat([df_oc_rs,df_new_oc_rs],ignore_index=True)
data_oc_rs.dropna(axis='columns',inplace=True)
data_oc_rs = pd.merge(data_oc_rs,df_fecha,on='id_fecha',how='inner')
data_oc_rs = pd.merge(data_oc_rs,df_razon_social,on='id_razon_social',how='inner')


In [235]:
data_oc_rs.shape

(396, 13)

In [234]:
data_oc_rs.to_csv(os.path.join(path_scoring_db,'data_oc_rs_sc.csv'),index=False)

## OEC CP

In [224]:
new_data_oec_cp = [] 
size_oec_cp = df_oec_cp.shape[0]
for i in range(300):
    rd_idx = randint(0,size_ci_cp-1)
    rd_f_idx = randint(0,size_new_fecha-1)
    id_rs_rd = df_oec_cp['id_razon_social'].iloc[rd_idx]
    id_cp_rd = df_oec_cp['id_curso_perfil'].iloc[rd_idx]
    id_fecha_rd = df_new_fecha['id_fecha'].iloc[rd_f_idx]
    new_data_oec_cp.append([id_rs_rd,id_cp_rd,id_fecha_rd])
    
   
df_new_oec_cp = pd.DataFrame(data=new_data_oec_cp,columns=['id_razon_social',	'id_curso_perfil',	'id_fecha'])
    

In [225]:
df_new_oec_cp = df_new_oec_cp.drop_duplicates()
df_new_oec_cp['id_fact_oec_cp'] = [x for x in range(df_oec_cp.shape[0],df_oec_cp.shape[0]+df_new_oec_cp.shape[0])]
df_new_oec_cp.insert(0, 'id_fact_oec_cp', df_new_oec_cp.pop('id_fact_oec_cp'))
data_oec_cp = pd.concat([df_oec_cp,df_new_oec_cp],ignore_index=True)
data_oec_cp.dropna(axis='columns',inplace=True)
data_oec_cp = pd.merge(data_oec_cp,df_curso_perfil,on='id_curso_perfil',how='inner')
data_oec_cp = pd.merge(data_oec_cp,df_fecha,on='id_fecha',how='inner')
data_oec_cp = pd.merge(data_oec_cp,df_razon_social,on='id_razon_social',how='inner')

In [226]:
data_oec_cp.shape

(18399, 21)

In [227]:
data_oec_cp.to_csv(os.path.join(path_scoring_db,'data_oec_cp_sc.csv'),index=False)

## OEC RS

In [229]:
df_new_oec_rs = df_new_oec_cp.groupby(['id_razon_social','id_fecha']).count().reset_index()
df_new_oec_rs['id_fact_oec_rs'] = [x for x in range(df_oec_rs.shape[0],df_oec_rs.shape[0]+df_new_oec_rs.shape[0])]
df_new_oec_rs.insert(0, 'id_fact_oec_rs', df_new_oec_rs.pop('id_fact_oec_rs'))
data_oec_rs = pd.concat([df_oec_rs,df_new_oec_rs],ignore_index=True)
data_oec_rs.dropna(axis='columns',inplace=True)
data_oec_rs = pd.merge(data_oec_rs,df_fecha,on='id_fecha',how='inner')
data_oec_rs = pd.merge(data_oec_rs,df_razon_social,on='id_razon_social',how='inner')


In [232]:
data_oec_rs.to_csv(os.path.join(path_scoring_db,'data_oec_rs_sc.csv'),index=False)

# Union de datos

In [11]:
filelist = os.listdir(path_scored_new_data)

In [12]:
filelist

['.ipynb_checkpoints',
 'sc_ci_cp_num_cap.csv',
 'sc_ci_cp_total_ganancias.csv',
 'sc_ci_cp_total_horas.csv',
 'sc_ci_rs_total_cursos.csv',
 'sc_ci_rs_volumen_cap.csv',
 'sc_ci_rs_volumen_ganancias.csv',
 'sc_ci_rs_volumen_horas.csv',
 'sc_oc_cp_num_cap_cer.csv',
 'sc_oc_cp_porcentaje_asistencia.csv',
 'sc_oc_cp_total_ganancias.csv',
 'sc_oc_cp_total_horas.csv',
 'sc_oc_rs_total_cursos.csv',
 'sc_oc_rs_volumen_cap_cer.csv',
 'sc_oc_rs_volumen_ganancias.csv',
 'sc_oc_rs_volumen_horas.csv',
 'sc_oec_cp_num_cer.csv',
 'sc_oec_cp_porcentaje_asistencia.csv',
 'sc_oec_cp_total_ganancias.csv',
 'sc_oec_cp_total_horas.csv',
 'sc_oec_rs_total_perfiles.csv',
 'sc_oec_rs_volumen_cer.csv',
 'sc_oec_rs_volumen_ganacias.csv',
 'sc_oec_rs_volumen_horas.csv',
 'sc_oec_rs_volumen_porcentaje.csv']

In [13]:
df = []
fact_oc_cp = None
fact_oec_cp = None
fact_ci_cp = None
fact_oc_rs = None
fact_oec_rs = None
fact_ci_rs = None
for file in filelist:
    if '.csv' in file:
            
        df = pd.read_csv(os.path.join(path_scored_new_data,file))
        idx_pred = [ i for i, word in enumerate(df.columns) if word.startswith('prediction') ]
        pred_col = df.columns[idx_pred][0]
        if 'oc_cp_' in file:
            if fact_oc_cp is None:
                fact_oc_cp = df
            else: 
                fact_oc_cp = fact_oc_cp.merge(df[['id_fact_oc_cp',pred_col]],on='id_fact_oc_cp',how='inner')

        elif 'oec_cp_' in file:
            if fact_oec_cp is None:
                fact_oec_cp = df
            else:
                fact_oec_cp = fact_oec_cp.merge(df[['id_fact_oec_cp',pred_col]],on='id_fact_oec_cp',how='inner')

        elif 'ci_cp_' in file:
            if fact_ci_cp is None:
                fact_ci_cp = df
            else:
                fact_ci_cp = fact_ci_cp.merge(df[['id_fact_ci_cp',pred_col]],on='id_fact_ci_cp',how='inner')

        elif 'oc_rs_' in file:
            if fact_oc_rs is None:
                fact_oc_rs = df
            else:
                fact_oc_rs = fact_oc_rs.merge(df[['id_fact_oc_rs',pred_col]],on='id_fact_oc_rs',how='inner')

        elif 'oec_rs_' in file:
            if fact_oec_rs is None:
                fact_oec_rs = df
            else:
                fact_oec_rs = fact_oec_rs.merge(df[['id_fact_oec_rs',pred_col]],on='id_fact_oec_rs',how='inner')

        elif 'ci_rs_' in file:
            if fact_ci_rs is None:
                fact_ci_rs = df
            else:
                fact_ci_rs = fact_ci_rs.merge(df[['id_fact_ci_rs',pred_col]],on='id_fact_ci_rs',how='inner')


In [14]:
fact_oc_cp['num_cap_cer']=0
fact_oc_cp['total_horas']=0
fact_oc_cp['total_ganancias']=0
fact_oc_cp['porcentaje_asistencia']=0

fact_oc_rs['total_cursos_perfiles']=0
fact_oc_rs['volumen_cap_cer']=0
fact_oc_rs['volumen_horas']=0
fact_oc_rs['volumen_ganancias']=0
fact_oc_rs['volumen_porcentaje_asistencia']=0


fact_oec_cp['num_cer']=0
fact_oec_cp['total_horas']=0
fact_oec_cp['total_ganancias']=0
fact_oec_cp['porcentaje_asistencia']=0

fact_oec_rs['total_perfiles']=0
fact_oec_rs['volumen_cer']=0
fact_oec_rs['volumen_horas']=0
fact_oec_rs['volumen_ganancias']=0
fact_oec_rs['volumen_porcentaje_asistencia']=0


fact_ci_cp['num_cap']=0
fact_ci_cp['total_horas']=0
fact_ci_cp['total_ganancias']=0

fact_ci_rs['total_cursos']=0
fact_ci_rs['volumen_cap']=0
fact_ci_rs['volumen_horas']=0
fact_ci_rs['volumen_ganancias']=0

In [15]:
fact_oc_cp_final = fact_oc_cp[['id_fact_oc_cp','id_razon_social','id_curso_perfil','id_fecha','num_cap_cer','prediction(num_cap_cer)','total_horas','prediction(total_horas)', 'total_ganancias', 'prediction(total_ganancias)', 'porcentaje_asistencia','prediction(porcentaje_asistencia)']]

fact_oc_rs_final = fact_oc_rs[['id_fact_oc_rs','id_razon_social','id_fecha','total_cursos_perfiles','prediction(total_cursos)','volumen_cap_cer','prediction(volumen_cap_cer)','volumen_horas','prediction(volumen_horas)', 'volumen_ganancias', 'prediction(volumen_ganancias)', 'volumen_porcentaje_asistencia']]

fact_oec_cp_final = fact_oec_cp[['id_fact_oec_cp','id_razon_social','id_curso_perfil','id_fecha','num_cer','prediction(num_cer)','total_horas','prediction(total_horas)', 'total_ganancias', 'prediction(total_ganancias)', 'porcentaje_asistencia','prediction(porcentaje_asistencia)']]

fact_oec_rs_final = fact_oec_rs[['id_fact_oec_rs','id_razon_social','id_fecha','total_perfiles','prediction(total_perfiles)','volumen_cer','prediction(volumen_cer)','volumen_horas','prediction(volumen_horas)', 'volumen_ganancias', 'prediction(volumen_ganancias)', 'volumen_porcentaje_asistencia','prediction(volumen_porcentaje_asistencia)']]

fact_ci_cp_final = fact_ci_cp[['id_fact_ci_cp','id_razon_social','id_curso_perfil','id_fecha','num_cap','prediction(num_cap)','total_horas','prediction(total_horas)', 'total_ganancias', 'prediction(total_ganancias)']]

fact_ci_rs_final = fact_ci_rs[['id_fact_ci_rs','id_razon_social','id_fecha','total_cursos','prediction(total_cursos)','volumen_cap','prediction(volumen_cap)','volumen_horas','prediction(volumen_horas)', 'volumen_ganancias', 'prediction(volumen_ganancias)']]

In [16]:
val_extra = fact_oc_cp_final.groupby(['id_razon_social','id_fecha']).sum().reset_index()[['id_razon_social','id_fecha','prediction(porcentaje_asistencia)']]

In [17]:
fact_oc_rs_final['prediction(volumen_porcentaje_asistencia)'] = val_extra['prediction(porcentaje_asistencia)']

C:\Users\sjime\AppData\Local\Temp\ipykernel_23088\4114751181.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_oc_rs_final['prediction(volumen_porcentaje_asistencia)'] = val_extra['prediction(porcentaje_asistencia)']


In [18]:
arr_df_finals = [fact_ci_cp_final.sort_values(by='id_fact_ci_cp'),fact_ci_rs_final.sort_values(by='id_fact_ci_rs'),fact_oc_cp_final.sort_values(by='id_fact_oc_cp'),fact_oc_rs_final.sort_values(by='id_fact_oc_rs'), fact_oec_cp_final.sort_values(by='id_fact_oec_cp'),fact_oec_rs_final.sort_values(by='id_fact_oec_rs')]
arr_df_finals_refact = []

In [19]:
for df_final in arr_df_finals:
    df_final_aux = df_final.copy()
    idxs_pred = [ i for i, word in enumerate(df_final_aux.columns) if word.startswith('prediction')]
    for i in idxs_pred:
        old_col = df_final_aux.columns[i]
        new_col = old_col.replace('(','_').replace(')','')
        df_final_aux = df_final_aux.rename(columns={old_col:new_col})
    arr_df_finals_refact.append(df_final_aux)

In [20]:
list_files_dwh = os.listdir(path_dwh_final)

In [21]:
df_oc_cp_dpt = pd.read_csv(os.path.join(path_dwh_final,'fact_oc_cp_dpt.csv'))

In [22]:
arr_df = []
for file in list_files_dwh:
    if 'fact' in file:
        df_old = pd.read_csv(os.path.join(path_dwh_final,file))
        arr_df.append(df_old)
        

In [23]:
doc_names = ['fact_ci_cp','fact_ci_rs','fact_oc_cp','fact_oc_rs','fact_oec_cp','fact_oec_rs']

In [24]:
for i in range(len(arr_df_finals_refact)):
    aux_1 = arr_df[i]
    aux_2 = arr_df_finals_refact[i]
    aux_2 = aux_2.iloc[aux_1.shape[0]:]
    doc_name = doc_names[i]
    df_dpt = pd.concat([aux_1,aux_2],ignore_index=True)
    df_dpt.to_csv(os.path.join(path_dwh_sc,doc_name+'_dpt_sc.csv'),index=False)

# Carga de datos

In [25]:
arr_tablas = []
file_list_dp = os.listdir(path_dwh_sc)
for file in file_list_dp:
    arr_tablas.append(pd.read_csv(os.path.join(path_dwh_sc,file)))

In [26]:
file_list_dp

['dim_curso_perfil.csv',
 'dim_fecha.csv',
 'dim_razon_social.csv',
 'fact_ci_cp_dpt_sc.csv',
 'fact_ci_rs_dpt_sc.csv',
 'fact_oc_cp_dpt_sc.csv',
 'fact_oc_rs_dpt_sc.csv',
 'fact_oec_cp_dpt_sc.csv',
 'fact_oec_rs_dpt_sc.csv']

In [27]:
df_ert = arr_tablas[0]
df_ert['area_familia']=df_ert['area_familia'].fillna('sin_area_familia')
df_ert['especialidad_sector']=df_ert['especialidad_sector'].fillna('sin_especialidad_sector')
df_ert['curso_perfil']=df_ert['curso_perfil'].fillna('curso_perfil')

In [52]:
arr_tablas[2]

,id_razon_social,razon_social,estado,tipo_razon_social,fecha_resolucion,canton,provincia
0,0,3cgreat solutions s.a.s.,calificado,operador de capacitación,29-11-2021,latacunga,cotopaxi
1,1,abalco haro cristian patricio,calificado,operador de capacitación,19-10-2021,quito,pichincha
2,2,abalco haro cristian patricio,habilitado,organismo evaluador de la conformidad,23-11-2021,guayaquil,guayas
3,3,abata carrera gloria beatriz,calificado,capacitador independiente,resolución no. mdt-ci-cal-2021-0021,esmeraldas,esmeraldas
4,4,abkrea ingenieria cia. ltda.,calificado,operador de capacitación,28-03-2021,ambato,tungurahua
...,...,...,...,...,...,...,...
2252,2252,zorrilla certuche mario andres,calificado,capacitador independiente,resolución no. mdt-ci-cal-2021-0467,quito,pichincha
2253,2253,zumarraga proaño bertha oliva,calificado,capacitador independiente,resolución no. mdt-ci-cal-2021-0264,quito,pichincha
2254,2254,zuriaga anchundia maria jose,calificado,capacitador independiente,resolución no. mdt-ci-cal-2021-0220,quevedo,los rios
2255,2255,zurita guevara jose roberto,calificado,operador de capacitación,29-07-2022,santo domingo,santo domingo de los tsachilas


In [29]:
# Conexión con MySQL
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='KappaTao1233'
)

mycursor = mydb.cursor()
# Creación de la base de datos¶
mycursor.execute("DROP DATABASE IF EXISTS dwh_setec")
mycursor.execute("CREATE DATABASE dwh_setec")
mycursor.execute("USE dwh_setec")
## DIMENSIÓN CURSO PERFIL
mycursor.execute("DROP TABLE IF EXISTS dim_curso_perfil")
mycursor.execute('''CREATE TABLE dim_curso_perfil (
id_curso_perfil INT PRIMARY KEY,
tipo VARCHAR(63) NOT NULL,
area_familia VARCHAR(255) NOT NULL,
especialidad_sector VARCHAR(255) NOT NULL,
curso_perfil VARCHAR(255) NOT NULL,
modalidad VARCHAR(255) NOT NULL,
carga_horaria INT NOT NULL,
costo FLOAT NOT NULL
)''')
sql_formula = 'INSERT INTO dim_curso_perfil ( id_curso_perfil, tipo, area_familia, especialidad_sector, curso_perfil, modalidad, carga_horaria, costo) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)'
for row in tqdm(df_ert.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()
## DIMENSION FECHA
mycursor.execute("DROP TABLE IF EXISTS dim_fecha")
mycursor.execute('''CREATE TABLE dim_fecha (
id_fecha INT PRIMARY KEY,
fecha VARCHAR(63) NOT NULL,
dia INT NOT NULL,
mes INT NOT NULL,
anio INT NOT NULL
)''')
sql_formula = 'INSERT INTO dim_fecha (id_fecha,fecha, dia, mes, anio) VALUES (%s,%s,%s,%s,%s)'
for row in tqdm(arr_tablas[1].values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()
## DIMENSIÓN RAZÓN SOCIAL
mycursor.execute("DROP TABLE IF EXISTS dim_razon_social")
mycursor.execute('''CREATE TABLE dim_razon_social (
id_razon_social INT PRIMARY KEY,
razon_social VARCHAR(255) NOT NULL,
estado VARCHAR(63) NOT NULL,
tipo_razon_social VARCHAR(255) NOT NULL,
fecha_resolucion VARCHAR(63) NOT NULL,
canton VARCHAR(63) NOT NULL,
provincia VARCHAR(63) NOT NULL
)''')
sql_formula = 'INSERT INTO dim_razon_social ( id_razon_social, razon_social,estado, tipo_razon_social,  fecha_resolucion, canton, provincia) VALUES (%s,%s,%s,%s,%s,%s,%s)'
for row in tqdm(arr_tablas[2].values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|██████████| 2257/2257 [00:01<00:00, 1365.83it/s]


In [41]:
# DATAMART CI
### CI CP
mycursor.execute("DROP TABLE IF EXISTS fact_ci_cp")
mycursor.execute('''CREATE TABLE fact_ci_cp (
id_fact_ci_cp INT PRIMARY KEY,
id_razon_social INT NOT NULL,
id_curso_perfil INT NOT NULL,
id_fecha INT NOT NULL,
num_cap INT NOT NULL,
prediction_num_cap INT NOT NULL,
total_horas INT NOT NULL,
prediction_total_horas INT NOT NULL,
total_ganancias FLOAT NOT NULL,
prediction_total_ganancias FLOAT NOT NULL
)''')
sql_formula = '''INSERT INTO fact_ci_cp (id_fact_ci_cp,id_razon_social,id_curso_perfil,id_fecha,num_cap,prediction_num_cap,total_horas,prediction_total_horas,total_ganancias,prediction_total_ganancias) 
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
print('work')
for row in tqdm(arr_tablas[3].astype('str').values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

work


100%|██████████| 1379/1379 [00:00<00:00, 1392.72it/s]


In [43]:


### CI RS
mycursor.execute("DROP TABLE IF EXISTS fact_ci_rs")
mycursor.execute('''CREATE TABLE fact_ci_rs (
id_fact_ci_rs INT PRIMARY KEY,
id_razon_social INT NOT NULL,
id_fecha INT NOT NULL,
total_cursos INT NOT NULL,
prediction_total_cursos INT NOT NULL,
volumen_cap INT NOT NULL,
prediction_volumen_cap INT NOT NULL,
volumen_horas INT NOT NULL,
prediction_volumen_horas INT NOT NULL,
volumen_ganancias FLOAT NOT NULL,
prediction_volumen_ganancias FLOAT NOT NULL
)''')
sql_formula = '''INSERT INTO fact_ci_rs (id_fact_ci_rs,
id_razon_social,
id_fecha,
total_cursos,
prediction_total_cursos,
volumen_cap,
prediction_volumen_cap,
volumen_horas,
prediction_volumen_horas,
volumen_ganancias,
prediction_volumen_ganancias)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
for row in tqdm(arr_tablas[4].astype('str').values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|██████████| 883/883 [00:00<00:00, 1350.55it/s]


In [45]:

## OC
### OC CP
mycursor.execute("DROP TABLE IF EXISTS fact_oc_cp")
mycursor.execute('''CREATE TABLE fact_oc_cp (
id_fact_oc_cp INT PRIMARY KEY,
id_razon_social INT NOT NULL,
id_curso_perfil INT NOT NULL,
id_fecha INT NOT NULL,
num_cap_cer INT NOT NULL,
prediction_num_cap_cer INT NOT NULL,
total_horas INT NOT NULL,
prediction_total_horas INT NOT NULL,
total_ganancias FLOAT NOT NULL,
prediction_total_ganancias FLOAT NOT NULL,
porcentaje_asistencia FLOAT NOT NULL,
prediction_porcentaje_asistencia FLOAT NOT NULL
)''')
sql_formula = '''INSERT INTO fact_oc_cp (id_fact_oc_cp,
id_razon_social,
id_curso_perfil,
id_fecha,
num_cap_cer,
prediction_num_cap_cer,
total_horas,
prediction_total_horas,
total_ganancias,
prediction_total_ganancias,
porcentaje_asistencia,
prediction_porcentaje_asistencia)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
for row in tqdm(arr_tablas[5].astype('str').values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|██████████| 2201/2201 [00:01<00:00, 1399.40it/s]


In [48]:

## OC RS
mycursor.execute("DROP TABLE IF EXISTS fact_oc_rs")
mycursor.execute('''CREATE TABLE fact_oc_rs (
id_fact_oc_rs INT PRIMARY KEY,
id_razon_social INT NOT NULL,
id_fecha INT NOT NULL,
total_cursos_perfiles INT NOT NULL,
prediction_total_cursos INT NOT NULL,
volumen_cap_cer INT NOT NULL,
prediction_volumen_cap_cer INT NOT NULL,
volumen_horas INT NOT NULL,
prediction_volumen_horas INT NOT NULL,
volumen_ganancias FLOAT NOT NULL,
prediction_volumen_ganancias FLOAT NOT NULL,
volumen_porcentaje_asistencia FLOAT NOT NULL,
prediction_volumen_porcentaje_asistencia FLOAT NOT NULL
)''')
sql_formula = '''INSERT INTO fact_oc_rs (id_fact_oc_rs,
id_razon_social,
id_fecha,
total_cursos_perfiles,
prediction_total_cursos,
volumen_cap_cer,
prediction_volumen_cap_cer,
volumen_horas,
prediction_volumen_horas,
volumen_ganancias,
prediction_volumen_ganancias,
volumen_porcentaje_asistencia,
prediction_volumen_porcentaje_asistencia)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
for row in tqdm(arr_tablas[6].astype('str').values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|██████████| 396/396 [00:00<00:00, 1337.74it/s]


In [49]:
## OEC
### OEC CP
mycursor.execute("DROP TABLE IF EXISTS fact_oec_cp")
mycursor.execute('''CREATE TABLE fact_oec_cp (
id_fact_oec_cp INT PRIMARY KEY,
id_razon_social INT NOT NULL,
id_curso_perfil INT NOT NULL,
id_fecha INT NOT NULL,
num_cer INT NOT NULL,
prediction_num_cer INT NOT NULL,
total_horas INT NOT NULL,
prediction_total_horas INT NOT NULL,
total_ganancias FLOAT NOT NULL,
prediction_total_ganancias FLOAT NOT NULL,
porcentaje_asistencia FLOAT NOT NULL,
prediction_porcentaje_asistencia FLOAT NOT NULL
)''')
sql_formula = '''INSERT INTO fact_oec_cp (id_fact_oec_cp,
id_razon_social,
id_curso_perfil,
id_fecha,
num_cer,
prediction_num_cer,
total_horas,
prediction_total_horas,
total_ganancias,
prediction_total_ganancias,
porcentaje_asistencia,
prediction_porcentaje_asistencia)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
for row in tqdm(arr_tablas[7].astype('str').values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|██████████| 18399/18399 [00:12<00:00, 1442.73it/s]


In [50]:


### OEC RS
for val in arr_tablas[8].columns:
    print(val)
mycursor.execute("DROP TABLE IF EXISTS fact_oec_rs")
mycursor.execute('''CREATE TABLE fact_oec_rs (
id_fact_oec_rs INT PRIMARY KEY,
id_razon_social INT NOT NULL,
id_fecha INT NOT NULL,
total_perfiles INT NOT NULL,
prediction_total_perfiles INT NOT NULL,
volumen_cer INT NOT NULL,
prediction_volumen_cer INT NOT NULL,
volumen_horas INT NOT NULL,
prediction_volumen_horas INT NOT NULL,
volumen_ganancias FLOAT NOT NULL,
prediction_volumen_ganancias FLOAT NOT NULL,
volumen_porcentaje_asistencia FLOAT NOT NULL,
prediction_volumen_porcentaje_asistencia FLOAT NOT NULL
)''')
sql_formula = '''INSERT INTO fact_oec_rs (id_fact_oec_rs,
id_razon_social,
id_fecha,
total_perfiles,
prediction_total_perfiles,
volumen_cer,
prediction_volumen_cer,
volumen_horas,
prediction_volumen_horas,
volumen_ganancias,
prediction_volumen_ganancias,
volumen_porcentaje_asistencia,
prediction_volumen_porcentaje_asistencia)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
for row in tqdm(arr_tablas[8].astype('str').values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

id_fact_oec_rs
id_razon_social
id_fecha
total_perfiles
prediction_total_perfiles
volumen_cer
prediction_volumen_cer
volumen_horas
prediction_volumen_horas
volumen_ganancias
prediction_volumen_ganancias
volumen_porcentaje_asistencia
prediction_volumen_porcentaje_asistencia


100%|██████████| 12839/12839 [00:08<00:00, 1428.62it/s]


In [51]:

## CREAR CLAVES FORANEAS OC
### FK dim fecha
mycursor.execute(''' 
ALTER TABLE fact_oc_cp
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha(id_fecha)
''')
mycursor.execute(''' 
ALTER TABLE fact_oc_rs
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha(id_fecha)
''')
### FK dim razon social
mycursor.execute(''' 
ALTER TABLE fact_oc_cp
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')
mycursor.execute(''' 
ALTER TABLE fact_oc_rs
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')
### FK dim curso perfil
mycursor.execute(''' 
ALTER TABLE fact_oc_cp
ADD FOREIGN KEY (id_curso_perfil) REFERENCES dim_curso_perfil(id_curso_perfil)
''')
## CREAR CLAVES FORANEAS OEC
### FK dim fecha
mycursor.execute(''' 
ALTER TABLE fact_oec_cp
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha(id_fecha)
''')
mycursor.execute(''' 
ALTER TABLE fact_oec_rs
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha(id_fecha)
''')
### FK dim razon social
mycursor.execute(''' 
ALTER TABLE fact_oec_cp
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')
mycursor.execute(''' 
ALTER TABLE fact_oec_rs
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')
### FK dim curso perfil
mycursor.execute(''' 
ALTER TABLE fact_oec_cp
ADD FOREIGN KEY (id_curso_perfil) REFERENCES dim_curso_perfil(id_curso_perfil)
''')
## CREAR CLAVES FORANEAS CI
### FK dim fecha
mycursor.execute(''' 
ALTER TABLE fact_ci_cp
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha(id_fecha)
''')
mycursor.execute(''' 
ALTER TABLE fact_ci_rs
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha(id_fecha)
''')
### FK dim razon social
mycursor.execute(''' 
ALTER TABLE fact_ci_cp
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')
mycursor.execute(''' 
ALTER TABLE fact_ci_rs
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')
### FK dim curso perfil
mycursor.execute(''' 
ALTER TABLE fact_ci_cp
ADD FOREIGN KEY (id_curso_perfil) REFERENCES dim_curso_perfil(id_curso_perfil)
''')